In [1]:
%run LanguageModel.py

In [2]:
%run DataLoader.py

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dl = DataLoader('train', ('de', 'en'), max_length = 50, device = device)

In [5]:
val_dl = DataLoader('dev', ('de', 'en'), languageModels = train_dl.languageModels, max_length = 50, device = device)

In [22]:
%run seq2seq.py

In [23]:
lm1 = train_dl.languageModels[train_dl.languages[0]]
lm2 = train_dl.languageModels[train_dl.languages[1]]
model_config = ModelConfig(input_size = lm1.n_tokens, hidden_size = 256, output_size = lm2.n_tokens, rnn_type='gru', bidirectional=False, attention = 'global', max_length=52)
training_config = TrainingConfig()
prediction_config = PredictionConfig(beam_width = 3)
s2s = seq2seq(model_config = model_config, training_config = training_config, prediction_config = prediction_config, device = device)

In [24]:
def loss_metric(input, output, ground_truth, nll):
    return nll / ground_truth[0].size(0)

def perplexity(input, output, ground_truth, nll):
    nll /= ground_truth[0].size(0)
    return math.exp(nll)

In [25]:
%run ScorePrinter.py

In [26]:
def train_epochs(epochs, print_every=1000):
    n_iters = len(train_dl)
    score_printer = ScorePrinter("Training", [('NLL', loss_metric),('Perplexity', perplexity)])
    
    for epoch in range(1, epochs+1):
        score_printer.startEpoch(epoch)
        idx_permutation = np.random.permutation(len(train_dl))[:5]
        for i, index in enumerate(idx_permutation):
            input_tensor, target_tensor = train_dl.tensorsFromPos(index)

            loss, output_sentence = s2s.train(input_tensor, target_tensor)
            score_printer.update(input_tensor, target_tensor, output_sentence, loss)
            
            if (i + 1) % print_every == 0:
                score_printer.printAvg(print_every)
                validate(print_every)
        
        score_printer.endEpoch(epoch)
        score_printer.printAvg(len(train_dl))
        validate()

In [27]:
import numpy as np
import math

def validate(n = None):
    score_printer = ScorePrinter("Validation", [('NLL', loss_metric),('Perplexity', perplexity)])
    n = n or len(val_dl)
    idx_permutation_val = np.random.permutation(len(val_dl))[:n]
    for j, val_index in enumerate(idx_permutation_val):
        input_tensor_val, target_tensor_val = val_dl.tensorsFromPos(val_index)
        loss, output_sentence = s2s.evaluate(input_tensor_val, target_tensor_val)
        score_printer.update(input_tensor_val, target_tensor_val, output_sentence, loss)
    score_printer.printAvg(showCount = False)

In [ ]:
train_epochs(50, print_every=2)



Epoch 1 started
 
[Training] 2 examples /  NLL: 10.698163951526988 Perplexity: 44447.673031596496  
[Validation]   NLL: 3.1205968316995873 Perplexity: 37.82237569450658  
[Training] 4 examples /  NLL: 10.53416652927151 Perplexity: 38184.223854993594  
[Validation]   NLL: 4.0439477641627475 Perplexity: 104.29563581908819 

Epoch 1 ended
 
[Training] 5 examples /  NLL: 10.350449852881493 Perplexity: 33546.71215984841  
[Validation]   NLL: 6.513326094814188 Perplexity: 1003.5231305283507 

Epoch 2 started
